In [1]:
import cv2
import numpy as np
import random

In [2]:
img=np.zeros((300,400), np.uint8)
dets=np.array([[83,54,165,163,0.8], [67,48,118,132,0.5], [91,38,192,171,0.6]], np.float)

In [3]:
img_cp=img.copy()
for box in dets.tolist():
    x1,y1,x2,y2,score=int(box[0]),int(box[1]),int(box[2]),int(box[3]),box[-1]
    y_text=int(random.uniform(y1, y2))
    cv2.rectangle(img_cp, (x1,y1), (x2, y2), (255, 255, 255), 2)
    cv2.putText(img_cp, str(score), (x2-30, y_text), 2,1, (255, 255, 0))
cv2.imshow("ori_img", img_cp)

In [4]:
def nms(dets, thresh):
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1) #所有box面积
    print "all box aeras: ", areas
    order = scores.argsort()[::-1] #降序排列得到scores的坐标索引

    keep = []
    while order.size > 0:
        i = order[0] #最大得分box的坐标索引
        keep.append(i)
        xx1 = np.maximum(x1[i], x1[order[1:]]) 
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]]) #最高得分的boax与其他box的公共部分(交集)

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1) #求高和宽，并使数值合法化
        inter = w * h #其他所有box的面积
        ovr = inter / (areas[i] + areas[order[1:]] - inter)  #IOU:交并比

        inds = np.where(ovr <= thresh)[0] #ovr小表示两个box交集少，可能是另一个物体的框，故需要保留
        order = order[inds + 1]  #iou小于阈值的框

    return keep

In [ ]:
rtn_box=nms(dets, 0.3)
cls_dets=dets[rtn_box, :]
print "nms box:", cls_dets

all box aress:  [ 9130.  4420. 13668.]
nms box: [[ 83.   54.  165.  163.    0.8]
 [ 67.   48.  118.  132.    0.5]]


In [ ]:
img_cp=img.copy()
for box in cls_dets.tolist():
    x1,y1,x2,y2,score=int(box[0]),int(box[1]),int(box[2]),int(box[3]),box[-1]
    y_text=int(random.uniform(y1, y2))
    cv2.rectangle(img_cp, (x1,y1), (x2, y2), (255, 255, 255), 2)
    cv2.putText(img_cp, str(score), (x2-30, y_text), 2,1, (255, 255, 0))
cv2.imshow("nms_img", img_cp)
cv2.waitKey(0)